In [1]:
import numpy as np
import pandas as pd
import torch
import os

In [ ]:
task = 'RE'
dataset = 'RE'
cluster_rank = torch.load(f'../DataSelection-IF/selection/{task}/{dataset}/Cluster-Rank.pkl',weights_only=False)
total_score = torch.load(f'../DataSelection-IF/selection/{task}/{dataset}/Total-Score.pkl',weights_only=False)

ppl_array = np.zeros(len(total_score))
for process_num in range(1,9,1): ## maximum of k process
    if os.path.exists('ppl/{}/{}/ppl-init-{}.csv'.format(task,dataset,process_num)): ## i-th gradient 
        ppl_df = pd.read_csv('ppl/{}/{}/ppl-init-{}.csv'.format(task,dataset,process_num),index_col=0)
        for index,row in ppl_df.iterrows():
            ppl_array[index] = row[0]

In [20]:
cluster_rank

{0: array([3506, 3472, 3482, 3493, 3467, 3494, 3498, 3476, 3477, 3486, 5337,
        3471, 3487, 3495, 3501, 3478, 3470, 3479, 3475, 3503, 3500, 3480,
        3504, 3492, 5336, 5333, 3489, 3485, 3483, 3484, 3499, 3468, 5331,
        3465, 3502, 3488, 3469, 5332, 3505, 3481, 3496, 3497, 3474, 3490,
        3473, 3491, 5330, 5334, 5339, 5338, 5340, 5335, 3466]),
 1: array([4605, 4592, 4601, 4598, 4597, 4606, 4593, 4588, 4610, 4590, 4595,
        4589, 4604, 4596, 4612, 4591, 4609, 4608, 4594, 4600, 4602, 4599,
        4613, 4607, 4585, 4587, 4603, 4611, 4586]),
 2: array([5098, 5097, 5090, 5083, 5079, 5077, 5082, 5078, 5080, 5084, 5081,
        5095, 5088, 5085, 5089, 6124, 5094, 5091, 5096, 6172, 5086, 5087,
        6125, 6123, 6126, 5092, 6170, 5093, 6171, 6173, 6297, 6298, 5601,
        5604, 5603, 5602]),
 3: array([ 146,  269,  435,  533,  531,  268,  529,  149,  441,  532,  145,
         398,   84,  492,  450,  568,  530,   80,  348,  570,  225,  418,
         571,  156,  347,  396

In [27]:
## reverse total score
import json
from src.util_func import load_yaml
task_list = [
    # ['RE','RE'],
    # ['ER','abt-buy'],
    # ['ER','semi-text-w'],
    # ['DC','hospital'],
    # ['DC','beer'],
    # ['DC','rayyan'],
    # ['DI','amazon'],
    # ['DI','walmart'],
    # ['AVE','oa_mine'],
    # # ['CTA','SimTab'],
    # ['CTA','WebTable'],
    # ['ER','walmart-amazon'],
    ['ER','semi-text-c'],
    # ['ER','amazon-google'],
    # ['ER','wdc'],
    # # ['SM','CMS']
]
for task,dataset in task_list:
    yaml_path = f'script/config_{task}_{dataset}.yaml'
    config = load_yaml(yaml_path)
    cluster_num = config['cluster_num']
    cluster_per_budget = config['sample_per_cluster']
    train_file_path = config['train_file_path']
    train_file = pd.read_json(train_file_path)

    cluster_rank = torch.load(f'../DataSelection-IF/selection/{task}/{dataset}/Cluster-Rank.pkl',weights_only=False)
    total_score = torch.load(f'../DataSelection-IF/selection/{task}/{dataset}/Total-Score.pkl',weights_only=False)

    ppl_array = np.zeros(len(total_score))
    for process_num in range(1,9,1): ## maximum of k process
        if os.path.exists('ppl/{}/{}/ppl-init-{}.csv'.format(task,dataset,process_num)): ## i-th gradient 
            ppl_df = pd.read_csv('ppl/{}/{}/ppl-init-{}.csv'.format(task,dataset,process_num),index_col=0)
            for index,row in ppl_df.iterrows():
                ppl_array[index] = row[0]

    QuRating_Score = np.load(f'QuRating/data/select_index_main_writing/{task}-{dataset}-QuRating-score.npy',allow_pickle=True).item()
    for i in range(len(total_score)):
        total_score[i] = total_score[i] - ppl_array[i]
        total_score[i] = total_score[i] + 6 * QuRating_Score[i]


    p2_choose_index = []
    count = 0
    for i in cluster_rank.keys():
        index_all = cluster_rank[i]
        sorted_score_all = total_score[index_all]
        sorted_index_all = index_all[np.argsort(-sorted_score_all)]
        # print(index_all,sorted_index_all)
        # break
        change = cluster_per_budget - len(set(index_all[:cluster_per_budget]).intersection(set(sorted_index_all[:cluster_per_budget])))
        count += change
        p2_choose_index.extend(sorted_index_all[:cluster_per_budget])
    select_IF_df = train_file.iloc[p2_choose_index]
    json.dump(select_IF_df.to_dict(orient='records'), open('train/{}/{}/train-select-w-main-QuRating.json'.format(task,dataset), 'w', encoding='utf-8'), ensure_ascii=False, indent=4)

/tmp/ipykernel_2440703/247758496.py:38: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ppl_array[index] = row[0]
/tmp/ipykernel_2440703/247758496.py:38: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ppl_array[index] = row[0]
/tmp/ipykernel_2440703/247758496.py:38: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ppl_array[index] = row[0]
/tmp/ipykernel_2440703/247758496.py:38: FutureWarning: Series.__getitem__ treating keys as positions

In [23]:
select_IF_df

,instruction,input,output,index
3506,You are an expert in relation extraction from ...,,"{""type"": ""University""}",453
3487,You are an expert in relation extraction from ...,,"{""type"": ""University""}",1668
5337,You are an expert in relation extraction from ...,,"{""type"": ""School""}",6753
3490,You are an expert in relation extraction from ...,,"{""type"": ""University""}",3655
5338,You are an expert in relation extraction from ...,,"{""type"": ""School""}",6753
...,...,...,...,...
6051,You are an expert in relation extraction from ...,,"{""type"": ""SnookerPlayer""}",3953
6294,You are an expert in relation extraction from ...,,"{""type"": ""SnookerChamp""}",3951
6050,You are an expert in relation extraction from ...,,"{""type"": ""SnookerPlayer""}",3953
6143,You are an expert in relation extraction from ...,,"{""type"": ""TableTennisPlayer""}",4177
